### Обработка данных 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from random import randint
import re

In [2]:
def caesar_cipher(text: str, k: int) -> str:
    encrypted_text = ""
    for s in text:
        order = new_order = ord(s)
        if s.isalpha():
            new_order = order + k
            if s.isupper():
                if new_order > ord("Я"):
                    new_order = ord('А') + order + k - ord('Я') - 1
            else:
                if new_order > ord("я"):
                    new_order = ord('а') + order + k - ord('я') - 1
        encrypted_text += chr(new_order)
    return encrypted_text

In [3]:
CHARS = list('абвгдежзийклмнопрстуфхцчшщъыьэюя ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [4]:
# все строки списками из индексов
def raw_to_indexes(lines: list[str]):
    indexes = torch.zeros((len(lines), MAX_LEN), dtype=int)
    for i in range(len(lines)):
        if raw_lines[i]:
            for j, w in enumerate(lines[i]):
                if j >= MAX_LEN: break
                indexes[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    return indexes

In [5]:
# обучающие данные
MAX_LEN = 30

# список всех строк
with open('datasets/onegin.txt', 'r', encoding='utf-8') as file:
    raw_lines = [re.sub(r'[^а-яА-Я ]*', '', line.strip().lower()) for line in file.readlines()]
    
# список всех строк (зашифрованный)
encrypted_raw_lines = [caesar_cipher(i, randint(1,10)) for i in raw_lines]

# все строки списками из индексов
indexes = raw_to_indexes(raw_lines)

# все зашифрованные строки списками из индексов
encrypted_indexes = raw_to_indexes(encrypted_raw_lines)

### Полученные тензоры данных

In [6]:
raw_lines[0:3]

['мой дядя самых честных правил',
 'когда не в шутку занемог',
 'он уважать себя заставил']

In [7]:
encrypted_raw_lines[0:3]

['ртн игиг хдрящ ыйхцсящ уфджмп',
 'лпдеб ож г щфулф ибожнпд',
 'рп хдвивфю узгб йвуфвдкн']

In [8]:
indexes[0:3]

tensor([[13, 15, 10, 33,  5, 32,  5, 32, 33, 18,  1, 13, 28, 22, 33, 24,  6, 18,
         19, 14, 28, 22, 33, 16, 17,  1,  3,  9, 12,  0],
        [11, 15,  4,  5,  1, 33, 14,  6, 33,  3, 33, 25, 20, 19, 11, 20, 33,  8,
          1, 14,  6, 13, 15,  4,  0,  0,  0,  0,  0,  0],
        [15, 14, 33, 20,  3,  1,  7,  1, 19, 29, 33, 18,  6,  2, 32, 33,  8,  1,
         18, 19,  1,  3,  9, 12,  0,  0,  0,  0,  0,  0]])

In [9]:
encrypted_indexes[0:3]

tensor([[17, 19, 14, 33,  9,  4,  9,  4, 33, 22,  5, 17, 32, 26, 33, 28, 10, 22,
         23, 18, 32, 26, 33, 20, 21,  5,  7, 13, 16,  0],
        [12, 16,  5,  6,  2, 33, 15,  7, 33,  4, 33, 26, 21, 20, 12, 21, 33,  9,
          2, 15,  7, 14, 16,  5,  0,  0,  0,  0,  0,  0],
        [17, 16, 33, 22,  5,  3,  9,  3, 21, 31, 33, 20,  8,  4,  2, 33, 10,  3,
         20, 21,  3,  5, 11, 14,  0,  0,  0,  0,  0,  0]])

In [10]:
indexes.shape, encrypted_indexes.shape # 1452 строки, длиной MAX_LEN символов

(torch.Size([1452, 30]), torch.Size([1452, 30]))

### Создание и обучение модели 

In [11]:
class Decryptor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decryptor, self).__init__()
        self.embedding = nn.Embedding(input_size, 64)
        self.rnn = nn.GRU(64, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        embedded = self.embedding(input)
        output, _ = self.rnn(embedded)
        output = self.linear(output)
        return output

In [12]:
input_size = len(CHAR_TO_INDEX)
hidden_size = 256
output_size = len(CHAR_TO_INDEX)

In [13]:
model = Decryptor(input_size, hidden_size, output_size)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
num_epochs = 200
for epoch in range(num_epochs):
    
    for i in range(int(len(indexes) /100)):
        X_batch = encrypted_indexes[i * 100 : (i + 1) * 100] # 100 x 20
        Y_batch = indexes[i * 100 : (i + 1) * 100]           # 100 x 20
        Y_batch = Y_batch.flatten()                          # 2000
        
        optimizer.zero_grad()
        
        output = model(X_batch)                              # 100 x 20 x 64
        output = output.view(-1, output_size)                # 2000 x 64
        
        loss = criterion(output, Y_batch)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 10, Loss: 1.1807951927185059
Epoch: 20, Loss: 0.8264318704605103
Epoch: 30, Loss: 0.5715091228485107
Epoch: 40, Loss: 0.41431865096092224
Epoch: 50, Loss: 0.27640530467033386
Epoch: 60, Loss: 0.3664447069168091
Epoch: 70, Loss: 0.16288608312606812
Epoch: 80, Loss: 0.1294758915901184
Epoch: 90, Loss: 0.11043868213891983
Epoch: 100, Loss: 0.09863854944705963
Epoch: 110, Loss: 0.0909227803349495
Epoch: 120, Loss: 0.08571521192789078
Epoch: 130, Loss: 0.08211541920900345
Epoch: 140, Loss: 0.07956685870885849
Epoch: 150, Loss: 0.0777130052447319
Epoch: 160, Loss: 0.07633092999458313
Epoch: 170, Loss: 0.07527927309274673
Epoch: 180, Loss: 0.07446476072072983
Epoch: 190, Loss: 0.07382403314113617
Epoch: 200, Loss: 0.07331325113773346


### Тестирование данных (обучающая выборка)

In [16]:
with torch.no_grad():
    output = model(encrypted_indexes)
    predicted_indexes = torch.argmax(output, dim=2)
    decrypted_indexes = predicted_indexes.numpy()

# Расшифрованные строки
decrypted_strings = []
for decrypted_index in decrypted_indexes:
    decrypted_string = ''.join([INDEX_TO_CHAR[i] for i in decrypted_index if i != 0])
    decrypted_strings.append(decrypted_string)

In [17]:
decrypted_strings[0:20]

['ной дядя самых честных правил',
 'иогда не в шутку занемог',
 'но уважать себя заставил',
 'в лучше выдумать не мог',
 'вво пример другим наука',
 'ио боже мой какая скука',
 'п больным сидеть и день и ночь',
 'не отходя ни шагу прочь',
 'иакое низкое коварство',
 'кнлуживого забавлять',
 'иму подушки поправлять',
 'иечально подносить лекарство',
 'вздыхать и думать про себя',
 'иогда же черт возьмет тебя',
 '',
 'пак думал молодой повеса',
 'плтя в пыли на почтовых',
 'исевышней волею зевеса',
 'наследник всех своих родных',
 'воузья людмилы и руслана']

### Тестирование данных (тестовая выборка - onegin_test)

In [18]:
# список всех строк
with open('datasets/onegin_test.txt', 'r', encoding='utf-8') as file:
    raw_lines = [re.sub(r'[^а-яА-Я ]*', '', line.strip().lower()) for line in file.readlines()]
    
# список всех строк (зашифрованный)
encrypted_raw_lines = [caesar_cipher(i, randint(1,10)) for i in raw_lines]

# все строки списками из индексов
indexes = raw_to_indexes(raw_lines)

# все зашифрованные строки списками из индексов
encrypted_indexes = raw_to_indexes(encrypted_raw_lines)

with torch.no_grad():
    output = model(encrypted_indexes)
    predicted_indexes = torch.argmax(output, dim=2)
    decrypted_indexes = predicted_indexes.numpy()

# Расшифрованные строки
decrypted_strings = []
for decrypted_index in decrypted_indexes:
    decrypted_string = ''.join([INDEX_TO_CHAR[i] for i in decrypted_index if i != 0])
    decrypted_strings.append(decrypted_string)
    
decrypted_strings[20:40]

['ну что ж ты едешь очень еаль',
 'чт оируюз ивкозеж ба лгйьеь иы',
 'нездихь ори хкннлец яху',
 'поедмет и мыслей и пера',
 'и слез и риул  ',
 'ноедртавь меня  ты чттишь  нет',
 ' д тад  лрежа из  чоть сейчас',
 'ини с охотой примут нас',
 'пнеден  поскакали други',
 'аяились ио састочены',
 'кнрой сазелые услуги',
 'востеприимоой старины',
 'песаз кигиуупый угрыенья',
 'несут на блюдечках варенья',
 'па столик ставят вошаной',
 'иуачил с бруплихмою водой',
 'пой еорогой самой краткой',
 'ионни летят во весь опор ',
 'пеперь подтлушаем украдиой',
 'вдроеа люших раздовор']